In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_n


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import os

# Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")

# 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfo)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 설정
template = """
주어진 학생의 자유 형식 자기소개 텍스트에서 다음 정보를 추출해줘.

{format_instructions}

자기소개:
{text}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 설정
# Groq API 키를 환경 변수에서 가져옵니다.
try:
    api_key = os.environ["OPENAI_API_KEY"]
except KeyError:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0  # 구조화된 데이터 추출에는 낮은 온도가 더 적합합니다.
)

# 체인 구성
chain = prompt | llm | parser

# 실행
student_text = """
안녕하세요! 저는 김민수이고 22살입니다. 
컴퓨터공학을 전공하고 있어요. 
취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 
앞으로 훌륭한 개발자가 되는 것이 목표입니다.
"""

structured_info = chain.invoke({"text": student_text})

# 결과 출력
print("--- 추출된 학생 정보 ---")
print(f"이름: {structured_info.name}")
print(f"나이: {structured_info.age}")
print(f"전공: {structured_info.major}")
print(f"취미: {', '.join(structured_info.hobbies)}")
print(f"목표: {structured_info.goal}")

print("\n--- 결과 타입 ---")
print(type(structured_info))

--- 추출된 학생 정보 ---
이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것

--- 결과 타입 ---
<class '__main__.StudentInfo'>
